# Subtitle's normalisation


Retrieve a text file using a regular expression for files.

In [24]:
import spacy
import os
import re
import module_traitement as m
# spacy.prefer_gpu()
# nlp = spacy.load("fr_dep_news_trf")
# from spacy.language import Language

In [25]:
# TEST - Pour Mediapi - LSF 
file_with_path = m.lister_fichiers_with_path("../data/aligned_mediapi/")
folder = m.lister_fichiers("../data/aligned_mediapi/")
output_seg = "test_new_seg_mediapi/"
output_cleaning = "test_cleaning_mediapi/"
#output_sent = "../data/sentence_matignon/"

In [26]:
def convertir_grand_nombre(nombre_texte):
    nombre_sans_points = re.sub(r'(\d)\.(\d)', r'\1\2', nombre_texte)
    return nombre_sans_points

def remplacer_points_adresses_email(texte):
    # Expression régulière pour repérer les adresses e-mail
    pattern = r'\b[\w.%+-]+@[\w.-]+\.[a-zA-Z]{2,}\b'

    # Fonction de remplacement pour remplacer les points par "POINT"
    def remplacer(match):
        return match.group().replace('.', 'POINT')

    texte_modifie = re.sub(pattern, remplacer, texte)
    return texte_modifie

def remplacer_points_adresses(texte):
    # Expression régulière pour repérer différentes formes d'adresses de site internet
    pattern = r'\b(?:https?://)?(?:www\.)?[\w.-]+\.[a-zA-Z]{2,}\b'

    # Fonction de remplacement pour remplacer les points par "POINT"
    def remplacer(match):
        return match.group().replace('.', 'POINT')

    texte_modifie = re.sub(pattern, remplacer, texte)
    return texte_modifie

def normaliser_points_de_suspension(texte):
    # Ajouter un espace après les points de suspension suivis d'une lettre
    texte_modifie = re.sub(r'\.\.\.(\w)', r'... \1', texte)

    # Retirer un espace avant les points de suspension précédés d'une lettre
    texte_modifie = re.sub(r'(\w)\s*\.\.\.', r'\1...', texte_modifie)

    return texte_modifie

def remplacer_ponctuation_html(texte):
    substitutions = [
        ("(", "&#40;"),
        (")", "&#41;"),
        ("?", "&#63;"),
        ("!", "&#33;"),
        (".", "&#46;"),
        ("...", "&#8230;")
    ]

    def remplacer_match(match):
        contenu_parentheses = match.group(1)
        for recherche, remplacement in substitutions:
            contenu_parentheses = contenu_parentheses.replace(recherche, remplacement)
        return f'({contenu_parentheses})'

    texte_modifie = re.sub(r'\(([^)]*)\)', remplacer_match, texte)
    return texte_modifie

def get_dict_vtt_clean(input):
    with open(input,encoding="utf-8") as f:
        lines = f.readlines()

    dict_sub = {}
    i = 0
    j = 0  

    while j < len(lines): 
        element = lines[j]
        if element.startswith("00:") or element.startswith("01:") or element.startswith("02:"):
            # Extraire le temps de début et de fin
            timing_line = element.strip().split(' --> ')
            start_time, end_time = timing_line

            text = ""
            while j + 1 < len(lines) and not lines[j + 1].startswith("00:") and not lines[j+1].startswith("01:") and not lines[j+1].startswith("02:"):
                j += 1
                content = lines[j]
                text = text + " " + content.strip()
                text = normaliser_points_de_suspension(text)
                text = text.replace("(...)","[SUSPENSIONP]") #régler ce problème.
                text = text.replace("....","[POINTS4]")
                text = remplacer_ponctuation_html(text)
                ### changement
                text = text.replace("…","...")
                text = text.replace("...","[SUSPENSION].") ### gérer la double suspension !
                text = re.sub(r'["“”«»]', '', text)
                text = text.replace("(???)","[INTERROGATION3]")
                text = text.replace("?!","[INTEREXCL].")
                text = text.replace("(?)","[INTERROGATION1]")
                text = text.replace("(??)","[INTERROGATION2]")
                text = text.replace("( ?)","[INTERROGATION1]")
                text = text.replace("?,","[INTERROGATION],")
                text = text.replace("!,","[EXCLAMATION],")
                text = text.replace("etc.,","[ETC],")
                text = text.replace("etc.)","[ETC])")
                text = text.replace("etc. .","[ETC].")
                text = text.replace("Etc","etc")
                text = text.replace("PAM !","[PAM]")
                text = text.replace("Média'Pi!","[NOM_MEDIA]")
                text = text.replace("Média'Pi !","[NOM_MEDIA]")
                text = text.replace("Media'Pi !","[NOM_MEDIA]")
                text = text.replace("Média'Pi&nbsp;!","[NOM_MEDIA]")
                text = text.replace("Média' Pi !","[NOM_MEDIA]")
                text = text.replace(".e.s","")
                text = text.replace(".ne.s","")
                text = text.replace(".e.","")
                text = text.replace(".e","")
                text = text.replace("!!","!")
                text = text.replace("??","?")
                #text = re.sub(r'\b(\w+)\s*\.\.\.\s*(\w+)\b', r'\1... \2', text)
                text = remplacer_points_adresses_email(text)
                text = remplacer_points_adresses(text)
                text = text.replace(".,","[POINT],")
                text = text.replace("y.a","y a")
                ### 
                text=text.replace("... -G. Attal : ","")
                text=text.replace("-G. Attal : ","")
                text=text.replace("G. Attal : ","")
                text = text.replace("-Bonjour", "Bonjour")
                text = text.replace("Bonjour.", "Bonjour,")
                text = text.replace(" M."," Monsieur")
                text = text.replace(" Mme"," Madame")
                text = text.replace("-"," ")
                text = convertir_grand_nombre(text)
                # text = text.replace("(","&#40;")
                # text = text.replace(")","&#41;")
                

            dict_sub[i] = {'start': start_time, 'end': end_time, 'text': text.strip()}
            i += 1

        j += 1

    return dict_sub


In [27]:
pattern = re.compile(r' [A-Z]\.')
for file,name in zip(file_with_path,folder):
    dict_sub = get_dict_vtt_clean(file)
    new_text = ""
    for k,v in dict_sub.items():
        for kk,vv in v.items():
            if kk == "text":
                new_text = new_text + vv.strip() + " "
    phrases = m.segmenter_texte_en_phrases(new_text)
    with open(f"test_sent_file_mediapi/{name}", 'w', encoding='utf-8') as f:
        for index, element in enumerate(phrases):
            f.write(element.strip())
            if index < len(phrases) - 1:
                f.write("\n")
